# Main Resource 

https://keras.io/examples/rl/actor_critic_cartpole/

# Secondary Resources 

Everything You Need To Master Actor Critic Methods
https://www.youtube.com/watch?v=LawaN3BdI00

https://github.com/philtabor/Youtube-Code-Repository/blob/master/ReinforcementLearning/PolicyGradient/actor_critic/tensorflow2/networks.py

https://github.com/sweetice/Deep-reinforcement-learning-with-pytorch

https://github.com/hermesdt/reinforcement-learning/blob/master/a2c/cartpole_a2c_episodic.ipynb



In [ ]:
# next steps are
# 1. Get training working per example in end game call back
# 2. Add CNN to start of network       - hopefully done 
# 3. Train with animations turned off

In [ ]:
#!pip install pygame

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install tensorflow_probability

In [1]:
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Conv2D, Conv1D, Flatten
from sklearn import preprocessing
import numpy as np
from keras.utils import to_categorical


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Random Moves 

In [ ]:
# import random
# import graphical, game

# def ai_callback(board, score, moves_left):
#     dir = random.randint(0, 1) == 0
#     return (random.randint(0, 7 if dir else 6), random.randint(0, 8 if dir else 9), dir)

# def transition_callback(board, move, score_delta, next_board, moves_left):
#     print(f"Score: {score_delta} , moves left: {moves_left} ")
#     #pass # This can be used to monitor outcomes of moves

# def end_of_game_callback(boards, scores, moves, final_score):
#     return False # True = play another, False = Done


# if __name__ == '__main__':
#     speedup = 1.0
#     g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
#     g.run()


# Creating the Action mapping dictionary 

In [3]:
cols = [0,1,2,3,4,5,6,7]

rows = [0,1,2,3,4,5,6,7,8,9]


# 0 = Horizontal swap  , 1 = vertical swap
dec = [0,1]

actions = []


for i in cols:
    for j in rows:
        for k in dec:
            
            # dont add furtherest right coloumn with swapping horizontal , as invalid 
            if i==7 and k==0:
                continue
            
            # if top row, dont add the swapping vertical choice, as invalid
            if j==0 and k==1:
                continue
            
            else:
                actions.append( (i,j,k) )
        
        
print(len(actions))

142


In [4]:
print(actions)

[(0, 0, 0), (0, 1, 0), (0, 1, 1), (0, 2, 0), (0, 2, 1), (0, 3, 0), (0, 3, 1), (0, 4, 0), (0, 4, 1), (0, 5, 0), (0, 5, 1), (0, 6, 0), (0, 6, 1), (0, 7, 0), (0, 7, 1), (0, 8, 0), (0, 8, 1), (0, 9, 0), (0, 9, 1), (1, 0, 0), (1, 1, 0), (1, 1, 1), (1, 2, 0), (1, 2, 1), (1, 3, 0), (1, 3, 1), (1, 4, 0), (1, 4, 1), (1, 5, 0), (1, 5, 1), (1, 6, 0), (1, 6, 1), (1, 7, 0), (1, 7, 1), (1, 8, 0), (1, 8, 1), (1, 9, 0), (1, 9, 1), (2, 0, 0), (2, 1, 0), (2, 1, 1), (2, 2, 0), (2, 2, 1), (2, 3, 0), (2, 3, 1), (2, 4, 0), (2, 4, 1), (2, 5, 0), (2, 5, 1), (2, 6, 0), (2, 6, 1), (2, 7, 0), (2, 7, 1), (2, 8, 0), (2, 8, 1), (2, 9, 0), (2, 9, 1), (3, 0, 0), (3, 1, 0), (3, 1, 1), (3, 2, 0), (3, 2, 1), (3, 3, 0), (3, 3, 1), (3, 4, 0), (3, 4, 1), (3, 5, 0), (3, 5, 1), (3, 6, 0), (3, 6, 1), (3, 7, 0), (3, 7, 1), (3, 8, 0), (3, 8, 1), (3, 9, 0), (3, 9, 1), (4, 0, 0), (4, 1, 0), (4, 1, 1), (4, 2, 0), (4, 2, 1), (4, 3, 0), (4, 3, 1), (4, 4, 0), (4, 4, 1), (4, 5, 0), (4, 5, 1), (4, 6, 0), (4, 6, 1), (4, 7, 0), (4, 7, 1)

### Convert to dictionary 

In [5]:
# set up keys
keys = [i for i in range(142)]

print(keys)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141]


In [6]:
dictOfActions = dict(zip(keys, actions))

In [7]:
print(dictOfActions)

{0: (0, 0, 0), 1: (0, 1, 0), 2: (0, 1, 1), 3: (0, 2, 0), 4: (0, 2, 1), 5: (0, 3, 0), 6: (0, 3, 1), 7: (0, 4, 0), 8: (0, 4, 1), 9: (0, 5, 0), 10: (0, 5, 1), 11: (0, 6, 0), 12: (0, 6, 1), 13: (0, 7, 0), 14: (0, 7, 1), 15: (0, 8, 0), 16: (0, 8, 1), 17: (0, 9, 0), 18: (0, 9, 1), 19: (1, 0, 0), 20: (1, 1, 0), 21: (1, 1, 1), 22: (1, 2, 0), 23: (1, 2, 1), 24: (1, 3, 0), 25: (1, 3, 1), 26: (1, 4, 0), 27: (1, 4, 1), 28: (1, 5, 0), 29: (1, 5, 1), 30: (1, 6, 0), 31: (1, 6, 1), 32: (1, 7, 0), 33: (1, 7, 1), 34: (1, 8, 0), 35: (1, 8, 1), 36: (1, 9, 0), 37: (1, 9, 1), 38: (2, 0, 0), 39: (2, 1, 0), 40: (2, 1, 1), 41: (2, 2, 0), 42: (2, 2, 1), 43: (2, 3, 0), 44: (2, 3, 1), 45: (2, 4, 0), 46: (2, 4, 1), 47: (2, 5, 0), 48: (2, 5, 1), 49: (2, 6, 0), 50: (2, 6, 1), 51: (2, 7, 0), 52: (2, 7, 1), 53: (2, 8, 0), 54: (2, 8, 1), 55: (2, 9, 0), 56: (2, 9, 1), 57: (3, 0, 0), 58: (3, 1, 0), 59: (3, 1, 1), 60: (3, 2, 0), 61: (3, 2, 1), 62: (3, 3, 0), 63: (3, 3, 1), 64: (3, 4, 0), 65: (3, 4, 1), 66: (3, 5, 0), 67: 

In [8]:
print(dictOfActions[5])

(0, 3, 0)


# My Attempt


In [9]:
# preprocess function for board 

def preprocess_string_int(input_string):
    
    """This will firstly convert the string representation into an interger representation. Then it will convert the interger
    representation of the board state into a tensor, to be evaluated in the neural net. """
    
    # removing the \n as they are really only used to make the text output of board state more readible 
    new_string = input_string.replace('\n', "")
    
    #stores ints representation of string
    inter = [] 
    
    for i in new_string:
        
        # a  = 1
        if i == 'a':
            inter.append(1)
        
        # b  = 2
        if i == 'b':
            inter.append(2)
            
        # c  = 3
        if i == 'c':
            inter.append(3)   
            
        # d  = 4
        if i == 'd':
            inter.append(4) 
            
        # '#' = 5
        if i == '#':
            inter.append(5) 
       
    
    # finally convert to tensor for neural nets
    inter_tf = tf.convert_to_tensor(inter)
    
    # make it ready for input into NN
    inter_tf = tf.expand_dims(inter_tf, 0)
    
    #one-hot encode board 
    inter_tf_ohe = to_categorical(inter_tf)
   
    
    return inter_tf_ohe

In [10]:
test = "abcd\nbcda#\nabcda"

In [11]:
y = preprocess_string_int(test)

print(y)

print(y.shape)

print(type(y))

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0.]]]
(1, 14, 6)
<class 'numpy.ndarray'>


# Set up of CNN Neural Network

In [12]:
num_actions = 142    # taking out some invalid moves

inputs = keras.Input(shape=(80,6,))

x=inputs

#cnn layers
x = keras.layers.Conv1D(32, kernel_size=3, name='Conv_1')(x)
x = keras.layers.LeakyReLU(0.1)(x)      

x = keras.layers.Conv1D(32, kernel_size=3, name='Conv_2')(x)
x = keras.layers.LeakyReLU(0.1)(x)


x = keras.layers.Conv1D(32, kernel_size=3, name='Conv_3')(x)
x = keras.layers.LeakyReLU(0.1)(x)

# flatten
x = keras.layers.Flatten(name='Flatten')(x)

# Fully connected hidden layers
x = keras.layers.Dense(1024, name='Dense_1')(x)
x = keras.layers.ReLU(name='ReLU_dense_1')(x)

x = keras.layers.Dense(512, name='Dense_2')(x)
x = keras.layers.ReLU(name='ReLU_dense_2')(x)

action = keras.layers.Dense(num_actions, activation='softmax', name='Output')(x)
critic = keras.layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs= [action, critic ])

# Set up of Neural Network

In [13]:
# # set up of AC model
# num_inputs = 80      # board state, theres 80 options 
# num_actions = 142    # taking out some invalid moves
# num_hidden_1 = 1024  # first hidden layer
# num_hidden_2 = 512   # second hidden layer 


# inputs = layers.Input(shape=(num_inputs,))

# #first hidden layer
# common_1 = layers.Dense(num_hidden_1, activation="relu")(inputs)

# #second hidden layer
# common_2 = layers.Dense(num_hidden_2, activation="relu")(common_1)

# # choosing action 
# action = layers.Dense(num_actions, activation="softmax")(common_2)
# # outputting critic score  
# critic = layers.Dense(1)(common_2)

# model = keras.Model(inputs=inputs, outputs=[action, critic])

#  Random Choices from AC NN 

Info from end game call back 

In [14]:
# Info run of AC NN 

# perfectly fine random 

import random
import graphical, game

action_probs_history = []
critic_value_history = []
scores_delta = []
scores_end = []
counter = 0
runs = 2

def ai_callback(board, score, moves_left):
    
    # pre-process board 
    integer_board = preprocess_string_int(board)
        
    # now actor critic to make decision of choice 
    action_probs, critic_val = model(integer_board)
    
    # storing critic val results
    critic_value_history.append(critic_val[0, 0])
    
    # Sample action from action probability distribution, higher probababilites more chance of getting selected
    action = np.random.choice(num_actions, p=np.squeeze(action_probs))
    
    # storing action probs results
    action_probs_history.append(tf.math.log(action_probs[0, action]))
    
    return dictOfActions[action]
            
    
def transition_callback(board, move, score_delta, next_board, moves_left):
    scores_delta.append(score_delta)
    print(f"Score: {score_delta} , moves left: {moves_left} ")
    #pass # This can be used to monitor outcomes of moves

    
    
def end_of_game_callback(boards, scores, moves, final_score):
    
    scores_end.append(scores)

    boards_ints = []

    for i in boards:
        boards_ints.append(preprocess_string_int(i))

    print("\nLENGTH BOARDS: " , len(boards))

    print("\nScores: " , scores)

    print("\nScore delta" , scores_delta)

    print("\nMoves: " ,  moves)

    print("\nFINAL SCORE: " , final_score)

    return False # True = play another, False = Done



if __name__ == '__main__':
          
    # repeat for x times , range(x) 
    
    for i in range(5):
        print("---------------------------------------------------------------------------------------------------")
        speedup = 10
        g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
        g.run()


--------------------------------------------------------------
Seed: 305171158494025747926995200363878327406
Score: -5 , moves left: 24 
Score: -5 , moves left: 23 
Score: -5 , moves left: 22 
Score: -5 , moves left: 21 
Score: -5 , moves left: 20 
Score: -5 , moves left: 19 
Score: 8 , moves left: 18 
Score: -5 , moves left: 17 
Score: 8 , moves left: 16 
Score: -5 , moves left: 15 
Score: -5 , moves left: 14 
Score: -5 , moves left: 13 
Score: 8 , moves left: 12 
Score: -5 , moves left: 11 
Score: -5 , moves left: 10 
Score: -5 , moves left: 9 
Score: -5 , moves left: 8 
Score: -5 , moves left: 7 
Score: -5 , moves left: 6 
Score: -5 , moves left: 5 
Score: -5 , moves left: 4 
Score: -5 , moves left: 3 
Score: -5 , moves left: 2 
Score: -5 , moves left: 1 
Score: -5 , moves left: 0 

LENGTH BOARDS:  26

Scores:  [0, -5, -10, -15, -20, -25, -30, -22, -27, -19, -24, -29, -34, -26, -31, -36, -41, -46, -51, -56, -61, -66, -71, -76, -81, -86]

Score delta [-5, -5, -5, -5, -5, -5, 8, -5, 8

# What

So we know the board throughout the 25 moves , running score of each move & individual score, and finally the move itself  

Also can keep a list of action probs and critic vales from ai_call back 

Need to train/update weights on this

# Trying to update weights using keras example

In [26]:
import random
import graphical, game

action_probs_history = []
critic_value_history = []
scores_delta = []

# Discount factor for past rewards
gamma = 0.99  

# Smallest number such that 1.0 + eps != 1.0
eps = np.finfo(np.float32).eps.item()  



def ai_callback(board, score, moves_left):
    
    # pre-process board 
    integer_board = preprocess_string_int(board)

    # now actor critic to make decision of choice 
    action_probs, critic_val = model(integer_board)
    
    # storing results
    critic_value_history.append(critic_val[0, 0])

    # Sample action from action probability distribution, higher probababilites more chance of getting selected
    action = np.random.choice(num_actions, p=np.squeeze(action_probs))
    # storing results
    action_probs_history.append(tf.math.log(action_probs[0, action]))
    
         
    return dictOfActions[action]
            
    
def transition_callback(board, move, score_delta, next_board, moves_left):
    scores_delta.append(score_delta)
    print(f"Score: {score_delta} , moves left: {moves_left} ")
    #pass # This can be used to monitor outcomes of moves

    
def end_of_game_callback(boards, scores, moves, final_score):
       
    running_reward = 0
    episode_reward = 0
    reward_history = [] 
    
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    huber_loss = keras.losses.Huber()
    


    with tf.GradientTape() as tape:


        rewards_history = scores
        episode_reward = final_score

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            
            #print("Discount sum: " , discounted_sum)
            
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()
        
        #print("RET: " , returns)


        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []


        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            
            #print("Diff: " , diff)
            
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )
                
        #print("crit loss: " ,critic_losses)

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        
        #print("Loss val:" , loss_value)
        
        #print("mod train vars: " ,model.trainable_variables)
        
        
        # these are none ! 
        grads = tape.gradient(loss_value, model.trainable_variables)
        
        # this is issue always none
        print("Grads : " , grads)
        
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()


    print("Maybe did something")
    return False # True = play another, False = Done


if __name__ == '__main__':
    speedup = 10
    g = graphical.Game(ai_callback, transition_callback, end_of_game_callback, speedup)
    g.run()


Seed: 269661543592309657617714846807087655832
Score: -5 , moves left: 24 
Score: -5 , moves left: 23 
Score: -5 , moves left: 22 
Score: -5 , moves left: 21 
Score: -5 , moves left: 20 
Score: -5 , moves left: 19 
Score: -5 , moves left: 18 
Score: -5 , moves left: 17 
Score: -5 , moves left: 16 
Score: -5 , moves left: 15 
Score: 16 , moves left: 14 
Score: -5 , moves left: 13 
Score: -5 , moves left: 12 
Score: -5 , moves left: 11 
Score: -5 , moves left: 10 
Score: -5 , moves left: 9 
Score: -5 , moves left: 8 
Score: -5 , moves left: 7 
Score: -5 , moves left: 6 
Score: -5 , moves left: 5 
Score: 8 , moves left: 4 
Score: 54 , moves left: 3 
Score: -5 , moves left: 2 
Score: -5 , moves left: 1 
Score: -5 , moves left: 0 
Loss val: tf.Tensor(-27.530937, shape=(), dtype=float32)
Grads :  [None, None, None, None, None, None, None, None, None, None, None, None, None, None]


ValueError: No gradients provided for any variable: ['Conv_1/kernel:0', 'Conv_1/bias:0', 'Conv_2/kernel:0', 'Conv_2/bias:0', 'Conv_3/kernel:0', 'Conv_3/bias:0', 'Dense_1/kernel:0', 'Dense_1/bias:0', 'Dense_2/kernel:0', 'Dense_2/bias:0', 'Output/kernel:0', 'Output/bias:0', 'dense/kernel:0', 'dense/bias:0'].